In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

# Set paths and parameters
data_dir = r'C:\Users\91808\Desktop\PROJECT 7th sem\brain_tumor_dataset\Training'  # For training data
validation_dir = r'C:\Users\91808\Desktop\PROJECT 7th sem\brain_tumor_dataset\Testing'  # For testing/validation data
classes = ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'no_tumor']
image_size = (224, 224)
batch_size = 32

# 1. Count Images in Each Class (Label Distribution)
label_counts = {label: len(os.listdir(os.path.join(data_dir, label))) for label in classes}
print("Label Counts:", label_counts)

# 2. Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.15  # Reserve 15% for validation
)

# 3. Data Generators for Training and Validation
# Training data generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


# 4. Compute Class Weights for Handling Imbalance
y_train = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# 5. Model Architecture with Transfer Learning (MobileNetV2)
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model layers

# Add custom layers for our specific classification task
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Four output classes
])

# 6. Compile the Model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# 7. Set Callbacks for Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# 8. Train the Model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr]
)

# 9. Evaluate the Model on Validation Set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# 10. Sample Augmentation Visualization
sample_batch = train_generator.next()
for i in range(5):
    plt.imshow(sample_batch[0][i])
    plt.title(f"Class: {np.argmax(sample_batch[1][i])}")
    plt.show()

# 11. Plot Training History
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Label Counts: {'glioma_tumor': 826, 'meningioma_tumor': 822, 'pituitary_tumor': 827, 'no_tumor': 395}
Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.
Class Weights: {0: 0.8686440677966102, 1: 0.8728710462287105, 2: 1.8164556962025316, 3: 0.8675937122128174}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 15s 2us/step
Epoch 1/50


C:\Users\91808\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


90/90 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.5903 - loss: 1.4340 - val_accuracy: 0.5838 - val_loss: 1.5168 - learning_rate: 1.0000e-04
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 61s 650ms/step - accuracy: 0.7829 - loss: 0.5363 - val_accuracy: 0.6168 - val_loss: 1.5622 - learning_rate: 1.0000e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 63s 671ms/step - accuracy: 0.8106 - loss: 0.4560 - val_accuracy: 0.6168 - val_loss: 1.5407 - learning_rate: 1.0000e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 66s 714ms/step - accuracy: 0.8387 - loss: 0.3988 - val_accuracy: 0.6497 - val_loss: 1.4432 - learning_rate: 1.0000e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 74s 787ms/step - accuracy: 0.8363 - loss: 0.3894 - val_accuracy: 0.6218 - val_loss: 1.6179 - learning_rate: 1.0000e-04
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 66s 697ms/step - accuracy: 0.8624 - loss: 0.3125 - val_accuracy: 0.6548 - val_loss: 1.8454 - learning_rate: 1.0000e-04
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 59s 627ms/step - accuracy: 0.8541

AttributeError: 'DirectoryIterator' object has no attribute 'next'